In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import sys
import collections
import cv2
import os
import math
import random
import pickle
import copy
import numpy as np
import pandas as pd
sys.path.append("..")

In [4]:
import torch
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse
from torch_geometric.data import DataLoader
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [5]:
from losses import BoxVAE_losses as loss
from evaluation import metrics
from utils import plot_utils
from utils import data_utils as data_loading
from components.DenseAutoencoder import DenseAutoencoder
from components.DenseAutoencoder import Decoder
from components.TwoStageAutoEncoder import TwoStageAutoEncoder

In [6]:
import matplotlib.pyplot as plt

In [8]:
# from mask_generation import masked_sketch

In [24]:
batch_size = 128
latent_dims = 64
num_nodes = 16
bbx_size = 4
num_classes = 7
label_shape = 1
nb_epochs = 1000
klw = loss.frange_cycle_linear(nb_epochs)
learning_rate = 0.000155
hidden1 = 32
hidden2 = 16
hidden3 = 128
dense_hidden1=8
dense_hidden2=4
adaptive_margin = True
fine_tune_box = False
output_log = False
area_encoding = False
run_prefix = "two_stage_small_obj_conditioning_fft_gcn_reccurent_refine_coarse_bbx_loss_hw_cosine"
variational=False
coupling = True
obj_bbx_conditioning = True
use_fft_on_bbx = True
use_gcn_in_decoder = True

In [8]:
batch_train_loader, batch_val_loader = data_loading.load_data(obj_data_postfix = '_obj_boundary',
                                                              part_data_post_fix = '_scaled',
                                                              file_postfix = '_combined',
                                                              seed=345,
                                                              batch_size=batch_size)

/opt/anaconda3/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [10]:
import gc
gc.collect()

0

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='mps')

In [40]:
del vae

NameError: name 'vae' is not defined

In [11]:
run_prefix

'two_stage_small_obj_conditioning_fft_gcn_reccurent_refine_coarse_bbx_loss_hw_cosine'

In [27]:
reconstruction_loss_arr = []
kl_loss_obj_arr = []
kl_loss_part_arr = []
bbox_loss_arr = []
refined_bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
refined_bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

bbox_loss_threshold = 1.0
vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                          use_fft_on_bbx,
                          use_gcn_in_decoder
                        )
vae.to(device)
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,300], gamma=0.75)
model_path = ('/Users/amrutamuthal/Documents/training_runs/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('/Users/amrutamuthal/Documents/training_runs/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
if not os.path.exists(model_path):
    os.makedirs(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_refined_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    IOU_weight_delta = torch.tensor([(1+epoch)/nb_epochs])
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        y_true = train_data.y
        class_true = y_true[:, :num_classes]
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        node_data_transformed = torch.cat(
            [node_data_true[:, :-2],
             node_data_true[:, -2:] -  node_data_true[:, -4:-2]], axis=-1)

        adj_true = train_data.edge_index
        batch = train_data.batch
        
        class_true  = torch.flatten(class_true)
        

        for param in vae.parameters():
            param.grad=None
        
        output = vae(
            adj_true,
            node_data_transformed,
            X_obj_true_transformed,
            label_true,
            class_true, variational, coupling, refine_iter=2
            # training=(epoch>100)
            )
        
        node_data_pred = output[0]
        node_data_pred_new = torch.cat(
            [node_data_pred[:, :, :-2],
             node_data_pred[:, :, -2:] + node_data_pred[:, :, -4:-2]], axis=-1)
        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0])-X_obj_pred)*torch.tensor([0.5]),
                                (torch.tensor([1.0])+X_obj_pred)*torch.tensor([0.5])),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]

        node_data_pred_refined = output[8]
        node_data_pred_refined_new = torch.cat(
            [node_data_pred_refined[:, :, :-2],
             node_data_pred_refined[:, :, -2:] + node_data_pred_refined[:, :, -4:-2]], axis=-1)

        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        bbox_loss_coarse = loss.coarse_bbx_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:])
        node_loss = loss.node_loss(label_pred,label_true)
        
        # reconstruction_loss = (node_loss)*num_nodes

        if use_gcn_in_decoder:
            refined_bbox_loss = loss.weighted_bbox_loss(
                pred_box=node_data_pred_refined_new,
                true_box=node_data_true[:, 1:],
                weight=IOU_weight_delta,
                margin=margin,
            )
            reconstruction_loss = (refined_bbox_loss + node_loss) * num_nodes # * epoch / nb_epochs
        
        else:
            refined_bbox_loss = torch.tensor([0.0])
        
        kl_weight = klw[icoef]
        if variational and (kl_weight>0):
            reconstruction_loss += kl_loss_part*kl_weight   
        
        if epoch >50:
            reconstruction_loss += bbox_loss_coarse

        reconstruction_loss.backward()
        
        optimizer.step()
        
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_refined_bbox_loss += refined_bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/refined_bbox_loss", batch_refined_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.item()/(i+1), global_step)
            
            
#     scheduler.step()
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_refined_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/refined', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach().item()/(i+1))
    kl_loss_obj_arr.append(batch_kl_loss_obj.detach().item()/(i+1))
    kl_loss_part_arr.append(batch_kl_loss_part.detach().item()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach().item()/(i+1))
    refined_bbox_loss_arr.append(batch_refined_bbox_loss.detach().item()/(i+1))
    node_loss_arr.append(batch_node_loss.detach().item()/(i+1))
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    
    batch_loss = torch.tensor([0.0])
    batch_kl_loss_part = torch.tensor([0.0])
    batch_kl_loss_obj = torch.tensor([0.0])
    batch_bbox_loss = torch.tensor([0.0])
    batch_refined_bbox_loss = torch.tensor([0.0])
    batch_obj_bbox_loss = torch.tensor([0.0])
    batch_node_loss = torch.tensor([0.0])
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        y_true = val_data.y
        class_true = torch.flatten(y_true[:, :num_classes])
        X_obj_true = y_true[:, num_classes:]
        X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
        node_data_transformed = torch.cat(
            [node_data_true[:, :-2],
             node_data_true[:, -2:] -  node_data_true[:, -4:-2]], axis=-1)
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        class_true  = torch.flatten(class_true)
        
        output = vae(adj_true, node_data_transformed, X_obj_true_transformed, label_true , class_true, variational, coupling)
        
        node_data_pred = output[0]
        node_data_pred_new = node_data_pred
        node_data_pred_new = torch.cat(
            [node_data_pred[:, :, :-2],
             node_data_pred[:, :, -2:] + node_data_pred[:, :, -4:-2]], axis=-1)

        X_obj_pred = output[1]
        X_obj_pred = torch.cat(((torch.tensor([1.0])-X_obj_pred)*torch.tensor([0.5]),
                                (torch.tensor([1.0])+X_obj_pred)*torch.tensor([0.5])),
                              -1)
        label_pred = output[2]
        z_mean_part = output[3]
        z_logvar_part = output[4]
        margin = output[5]
        z_mean_obj = output[6]
        z_logvar_obj = output[7]

        node_data_pred_refined = output[8]
        node_data_pred_refined_new = node_data_pred_refined
        node_data_pred_refined_new = torch.cat(
            [node_data_pred_refined_new[:, :, :-2],
             node_data_pred_refined_new[:, :, -2:] + node_data_pred_refined_new[:, :, -4:-2]], axis=-1)
        
        
        obj_bbox_loss = loss.obj_bbox_loss(pred_box=X_obj_pred, true_box=X_obj_true, weight=IOU_weight_delta, has_mse=False)
        kl_loss_obj = loss.kl_loss(z_mean_obj, z_logvar_obj)
        kl_loss_part = loss.kl_loss(z_mean_part, z_logvar_part)
        bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        bbox_loss_coarse = loss.coarse_bbx_loss(pred_box=node_data_pred_new, true_box=node_data_true[:,1:])
        refined_bbox_loss = loss.weighted_bbox_loss(pred_box=node_data_pred_refined_new, true_box=node_data_true[:,1:], weight=IOU_weight_delta, margin=margin)
        node_loss = loss.node_loss(label_pred,label_true)
        
        if kl_weight>0:
            reconstruction_loss = kl_loss_part*kl_weight + (bbox_loss + node_loss)*num_nodes
        else:
            reconstruction_loss = (refined_bbox_loss + node_loss)*num_nodes

        if epoch > 50:
            reconstruction_loss +=  refined_bbox_loss
            
        batch_loss += reconstruction_loss
        batch_kl_loss_part += kl_loss_part
        batch_kl_loss_obj += kl_loss_obj
        batch_bbox_loss += bbox_loss
        batch_refined_bbox_loss += refined_bbox_loss
        batch_obj_bbox_loss += obj_bbox_loss
        batch_node_loss += node_loss
    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[:num_nodes,1:5]*label_true[:num_nodes]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred_refined_new[0]*label_true[:num_nodes]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/refined', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss.detach().item()/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss.detach().item()/(i+1))
    node_loss_val_arr.append(batch_node_loss.detach().item()/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_part", batch_kl_loss_part.item()/(i+1), global_step)
    writer.add_scalar("Loss/train/kl_loss_obj", batch_kl_loss_obj.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/refined_bbox_loss", batch_refined_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/obj_bbox_loss", batch_obj_bbox_loss.item()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.item()/(i+1), global_step)
       
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')
        
    if ((kl_loss_part_arr[-1]>0.5) and 
        (abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.07) and 
        (bbox_loss_arr[-1]<bbox_loss_threshold) and (epoch>300)):
        
        icoef = icoef + 1
        bbox_loss_threshold*=0.9

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(min(100,int(len(node_data_true)/num_nodes))):    
    image = plot_utils.plot_bbx(np.reshape((node_data_true[num_nodes*i:num_nodes*(i+1),1:5]).detach().to("cpu").numpy(),
                                    image_shape)).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-input', image, global_step, dataformats='HWC')
    image = plot_utils.plot_bbx((node_data_pred[i]*label_true[num_nodes*i:num_nodes*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/'+str(i)+'-generated', image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

RuntimeError: Tensor for argument input is on cpu but expected on mps

In [21]:
label_pred

tensor([[[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        ...,

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]],

        [[nan],
         [nan],
         [nan],
         ...,
         [nan],
         [nan],
         [nan]]], grad_fn=<ViewBackward0>)

In [62]:
 sum(p.numel() for p in vae.parameters() if p.requires_grad)

173005

In [26]:
#testing loop
model_path = ('/Users/amrutamuthal/Documents/training_runs/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('/Users/amrutamuthal/Documents/training_runs/run/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = False
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                          use_fft_on_bbx,
                          use_gcn_in_decoder,
                        )

vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

# decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    X_obj_true_transformed = X_obj_true[:,2:]-X_obj_true[:,:2]
    node_data_transformed = torch.cat(
            [node_data_true[:, :-2],
             node_data_true[:, -2:] -  node_data_true[:, -4:-2]], axis=-1)
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = vae(adj_true, node_data_transformed, X_obj_true_transformed, label_true , class_true, variational, coupling, refine_iter=2)
    node_data_pred_test = output[0]
    node_data_pred_test_refined = output[8]
    node_data_pred_test_refined_new = torch.clip(torch.cat(
            [node_data_pred_test_refined[:, :, :-2],
             node_data_pred_test_refined[:, :, -2:] + node_data_pred_test_refined[:, :, -4:-2]], axis=-1), min=0.0, max=1.0)
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    X_obj_pred_test = torch.cat(((torch.tensor([1.0])-X_obj_pred_test)*torch.tensor([0.5]),
                                (torch.tensor([1.0])+X_obj_pred_test)*torch.tensor([0.5])),
                              -1)
    res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test_refined_new,
                               X_obj_pred_test,
                               label_true,
                               class_true,
                               num_nodes,
                               num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            
            image = plot_bbx(node_data_true[j].detach().to("mps").numpy().astype(float))/255
            pred_image = plot_bbx(node_data_pred_test[j].detach().to("mps").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

# rel_metrics.csv')sult.to_csv(model_path+'/raw_metrics.csv')
# res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
# res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
# result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_leve

/var/folders/ww/3nytfk1917b8nz7dchtckxnm0000gn/T/ipykernel_18489/3865254239.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vae.load_state_dict(torch.load(model_path+ '

FileNotFoundError: [Errno 2] No such file or directory: '/Users/amrutamuthal/Documents/training_runs/model/two_stage_small_obj_conditioning_fft_gcn_reccurent_refine_coarse_bbx_loss_hw_cosine/Obj-Box-0.000155-batch-128-h1-32-h2-16-h3-128-test/model_weights.pth'

In [25]:
node_data_pred_test_refined[0]

tensor([[0.0017, 0.0054, 0.1568, 0.2406],
        [0.0026, 0.0035, 0.0281, 0.0269],
        [0.0026, 0.0035, 0.0281, 0.0269],
        [0.1431, 0.0410, 0.6509, 0.5488],
        [0.0587, 0.0413, 0.2518, 0.3063],
        [0.2748, 0.5259, 0.0651, 0.1780],
        [0.2688, 0.6391, 0.0692, 0.1424],
        [0.0027, 0.0036, 0.0283, 0.0271],
        [0.3561, 0.5813, 0.0665, 0.1637],
        [0.6355, 0.5541, 0.0996, 0.1835],
        [0.7963, 0.8297, 0.0997, 0.1775],
        [0.6308, 0.5922, 0.0796, 0.1644],
        [0.4324, 0.6743, 0.0947, 0.2553],
        [0.7592, 0.2521, 0.2495, 0.3865],
        [0.7295, 0.9013, 0.0467, 0.0531],
        [0.4062, 0.9113, 0.0437, 0.0471]], grad_fn=<SelectBackward0>)

In [27]:
node_data_true[:16]

tensor([[ 1.0000,  0.3237,  0.3511,  0.4058,  0.4101],
        [ 0.0000,  0.3237,  0.3511,  0.3237,  0.3511],
        [ 0.0000,  0.3237,  0.3511,  0.3237,  0.3511],
        [ 4.0000,  0.3597,  0.3885,  0.5900,  0.5411],
        [ 5.0000,  0.3583,  0.3870,  0.4403,  0.4748],
        [ 6.0000,  0.3899,  0.5267,  0.4086,  0.5785],
        [ 7.0000,  0.3942,  0.5756,  0.4389,  0.6159],
        [ 0.0000,  0.3237,  0.3511,  0.3237,  0.3511],
        [ 9.0000,  0.4144,  0.6159,  0.4345,  0.6505],
        [10.0000,  0.5123,  0.5310,  0.5914,  0.5742],
        [11.0000,  0.5295,  0.5655,  0.5914,  0.6145],
        [12.0000,  0.5310,  0.5583,  0.5555,  0.5886],
        [13.0000,  0.4892,  0.5857,  0.5425,  0.6418],
        [14.0000,  0.5871,  0.4504,  0.6764,  0.5497],
        [15.0000,  0.5295,  0.5986,  0.5569,  0.6145],
        [16.0000,  0.4878,  0.6317,  0.5065,  0.6418]])

In [20]:
result[['obj_class', 'part_labels', 'IOU', 'MSE', 'id']].groupby(['obj_class', 'part_labels']).mean().reset_index()

,obj_class,part_labels,IOU,MSE,id
0,bird,head,0.634835,0.000314,3407.099838
1,bird,lfoot,0.228812,0.000908,3286.593052
2,bird,lleg,0.350625,0.000589,3295.105622
3,bird,lwing,0.410575,0.001764,3419.445378
4,bird,neck,0.531832,0.000306,3395.612371
...,...,...,...,...,...
81,sheep,lhorn,0.280637,0.001024,3229.114865
82,sheep,neck,0.711767,0.000159,3284.448889
83,sheep,rhorn,0.309173,0.000749,3251.960265
84,sheep,tail,0.239571,0.001743,3481.894198


In [21]:
res_obj_level = result[['obj_class', 'IOU', 'MSE', 'id']].groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]


,obj_class,IOU,MSE
0,bird,0.567654,0.000499
1,cat,0.640374,0.001215
2,cow,0.548553,0.000792
3,dog,0.501625,0.000916
4,horse,0.424601,0.001495
5,person,0.525912,0.001345
6,sheep,0.699196,0.000332


In [ ]:
def inference(X_obj, part_decoder, obj_class, nodes, batch_size, latent_dims):
    
    nodes = torch.reshape(nodes,(batch_size,part_decoder.num_nodes))
    obj_class = torch.reshape(obj_class, 
                              (batch_size, num_classes))
    
    # obj sampling
    z_latent_obj = X_obj
    
    print(z_latent_obj.shape, obj_class.shape)
    conditioned_obj_latent = torch.cat([obj_class, z_latent_obj],dim=-1)
    conditioned_obj_latent = torch.cat([nodes, conditioned_obj_latent],dim=-1)

    # part sampling
    z_latent_part = torch.normal(torch.zeros([batch_size,latent_dims]))
    conditioned_part_latent = torch.cat([conditioned_obj_latent, z_latent_part],dim=-1)
    
    x_bbx, _, _, _ = part_decoder(conditioned_part_latent)
    
    return x_bbx, X_obj
    

In [ ]:
#testing loop
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')


write_tensorboard = True
if write_tensorboard:
    writer = SummaryWriter(summary_path)

vae = TwoStageAutoEncoder(latent_dims,
                          num_nodes,
                          bbx_size,
                          num_classes,
                          label_shape,
                          hidden1,
                          hidden2,
                          hidden3,
                          dense_hidden1,
                          dense_hidden2,
                          adaptive_margin,
                          output_log,
                          area_encoding,
                          coupling,
                          obj_bbx_conditioning,
                        )


vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

part_decoder = vae.gcn_decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
class_dict = {0:'cow', 1:'sheep', 2:'bird', 3:'person', 4:'cat', 5:'dog', 6:'horse'}
res_dfs = []
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    y_true = val_data.y
    class_true = y_true[:, :num_classes]
    X_obj_true = y_true[:, num_classes:]
    adj_true = val_data.edge_index
    class_true  = torch.flatten(class_true)
    
    output = inference(X_obj_true, part_decoder, class_true, label_true, batch_size, latent_dims)
    node_data_pred_test = output[0]
    X_obj_pred_test = output[1]
#     node_data_pred_test, X_obj_pred_test, label_pred, z_mean, z_logvar, margin = output
    
#     res_dfs.append(metrics.get_metrics(node_data_true, X_obj_true, node_data_pred_test,
#                                X_obj_pred_test,
#                                label_true,
#                                class_true,
#                                num_nodes,
#                                num_classes))
    
    if write_tensorboard:
        
        for j in range(int(len(node_data_true)/num_nodes)):
            image = plot_utils.plot_bbx((node_data_true[num_nodes*j:num_nodes*(j+1)
                                                       ,1:5]*label_true[num_nodes*j:num_nodes*(j+1)]).detach().to("mps").numpy().astype(float))/255
            pred_image = plot_utils.plot_bbx((node_data_pred_test[j]*label_true[num_nodes*j:num_nodes*(j+1)]).detach().to("mps").numpy()).astype(float)/255

            writer.add_image('test_result/images/'+str(j)+'-input/', image, global_step, dataformats='HWC')  
            writer.add_image('test_result/images/'+str(j)+'-generated/', pred_image, global_step, dataformats='HWC')
            

result = pd.concat(res_dfs)
result['obj_class'] = np.where(result['obj_class'].isna(), 0, result['obj_class'])
result['obj_class'] = result['obj_class'].astype('int')
result['obj_class'].replace(class_dict, inplace=True)
result.where(result['part_labels']!=0, np.NaN, inplace=True)
result['part_labels'] = np.where(result['part_labels'].isna(), 0, result['part_labels'])
result['part_labels'] = result['part_labels'].astype('int')
result['id'] = np.repeat(np.array(list(range(int(len(result)/num_nodes)))), 16)

if write_tensorboard:
    writer.flush()
    writer.close()

bird_labels = {'head':1 , 'torso':2, 'neck':3, 'lwing':4, 'rwing':5, 'lleg':6, 'lfoot':7, 'rleg':8, 'rfoot':9, 'tail':10}
cat_labels = {'head':1, 'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12}
cow_labels = {'head':1,'lhorn':2, 'rhorn':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14}
dog_labels = {'head':1,'torso':2, 'neck':3, 'lfleg':4, 'lfpa':5, 'rfleg':6, 'rfpa':7, 'lbleg':8, 'lbpa':9, 'rbleg':10, 'rbpa':11, 'tail':12, 'muzzle':13}
horse_labels = {'head':1,'lfho':2, 'rfho':3, 'torso':4, 'neck':5, 'lfuleg':6, 'lflleg':7, 'rfuleg':8, 'rflleg':9, 'lbuleg':10, 'lblleg':11, 'rbuleg':12, 'rblleg':13, 'tail':14, 'lbho':15, 'rbho':16}
person_labels = {'head':1, 'torso':2, 'neck': 3, 'llarm': 4, 'luarm': 5, 'lhand': 6, 'rlarm':7, 'ruarm':8, 'rhand': 9, 'llleg': 10, 'luleg':11, 'lfoot':12, 'rlleg':13, 'ruleg':14, 'rfoot':15}
sheep_labels = cow_labels
part_labels_combined_parts = {'bird': bird_labels, 'cat': cat_labels, 'cow': cow_labels, 'dog': dog_labels, 'sheep': sheep_labels, 'horse':horse_labels,'person':person_labels}

for k, v in part_labels_combined_parts.items():
    new_map = {}
    for pk, pv in v.items():
        new_map[pv]=pk
    part_labels_combined_parts[k] = new_map
    
for k, v in part_labels_combined_parts.items():
    result.loc[result['obj_class']==k, ['part_labels']] = result.loc[result['obj_class']==k,['part_labels']].replace(v).copy()

result.to_csv(model_path+'/raw_metrics.csv')
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']].to_csv(model_path+'/obj_level_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',  'IOU', 'MSE']].to_csv(model_path+'/part_level_metrics.csv')

In [ ]:
model_path = ('D:/meronym_data/model/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')
summary_path = ('D:/meronym_data/runs/'+run_prefix+'/Obj-Box-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3)+'-test')

result = pd.read_csv(model_path+'/raw_metrics.csv')
result.groupby(['obj_class', 'part_labels']).mean().reset_index()[['obj_class', 'part_labels',
                                                                   'IOU', 'MSE']]

In [77]:
res_obj_level = result.groupby(['obj_class', 'id']).mean().reset_index()
res_obj_level.groupby(['obj_class']).mean().reset_index()[['obj_class', 'IOU', 'MSE']]

TypeError: agg function failed [how->mean,dtype->object]

In [22]:
res_obj_level[['IOU', 'MSE']].mean()

IOU    0.554299
MSE    0.000972
dtype: float64

In [ ]:
X_obj_pred[:10,2:]-X_obj_pred[:10,:2]

In [ ]:
X_obj_true[:10, 2:]-X_obj_true[:10, :2]

In [ ]:
res_obj_level.to_csv(model_path+'/obj_metrics.csv')

In [ ]:
result.mean().reset_index()

In [ ]:
outfile = 'D:/meronym_data/generate_boxes.npy'
with open(outfile, 'wb') as pickle_file:
    pred_boxes = np.concatenate(pred_boxes)
    pickle.dump(pred_boxes, pickle_file)
outfile = 'D:/meronym_data/generate_boxesobj_class.npy'
with open(outfile, 'wb') as pickle_file:
    pickle.dump(classes,pickle_file)